In [ ]:
"""
Clip Raster using all shapes in one folder
"""

in_rst = '/home/jasp/datasets/mdt_m888/rst_ref50m.tif'
clip_fld = '/home/jasp/datasets/mdt_m888/rst_masks'
out_fld = '/home/jasp/datasets/mdt_m888/mdt50_masks'

import os
from glass.pys.oss import lst_ff, fprop
from glass.g.rst.ovl import clip_rst

shps = lst_ff(clip_fld, file_format='.shp')

for shp in shps:
    clip_rst(in_rst, shp, os.path.join(out_fld, fprop(shp, 'fn') + '.tif'))

In [ ]:
"""
Clip all rasters in one folder using matching files in
other folder as clipping raster

Relation between input rasters and clip rasters are stablished
using a ID in the end of the files
"""

in_rsts   = '/home/jasp/mrgis/dem_work/dem_eu_cubic/'
clip_rsts = '/home/jasp/mrgis/dem_work/ref_parts/'
out_rsts  = '/home/jasp/mrgis/dem_work/eudem_bspline'
basename  = 'asterdem'

In [ ]:
import os
import pandas as pd
import multiprocessing as mp
from glass.pys.oss import lst_ff, fprop, cpu_cores
from glass.ng.pd.split import df_split
from glass.g.rst.ovl import grscliprst

In [ ]:
# List input rasters
rsts = [{
    "fid" : int(r.split('.')[0].split('_')[-1]),
    'rst' : r
} for r in lst_ff(in_rsts, rfilename=True, file_format='.tif')]

# List Clip Rasters
clips = [{
    "jfid" : int(cr.split('.')[0].split('_')[-1]),
    'clipr' : cr
} for cr in lst_ff(clip_rsts, rfilename=True, file_format='.tif')]

# Get match between input rasters and clip rasters
# Use ID at end of the files
df = pd.DataFrame(rsts)
jdf = pd.DataFrame(clips)

df = df.merge(jdf, how='left', left_on='fid', right_on='jfid')

In [ ]:
# Clip Rasters using multiprocessing approach

n_cpu = cpu_cores()
dfs = df_split(df, n_cpu)

def th_clip_rst(_df):
    for idx, row in _df.iterrows():
        grscliprst(
            os.path.join(in_rsts, str(row.rst)),
            os.path.join(clip_rsts, str(row.clipr)),
            os.path.join(out_rsts, basename + '_' + str(row.fid) + '.tif')
        )

ths = [mp.Process(
    target=th_clip_rst, name='th_' + str(i),
    args=(dfs[i],)
) for i in range(len(dfs))]

In [ ]:
for t in ths:
    t.start()

for t in ths:
    t.join()